this part of the code is where i:

- [ ] import needed data
- [ ] encode the categorical data and convert them to floats 
- [ ] use lazypredict to determine the best options
- [ ] train model
- [ ] hyperparameter tunning
- [ ] model logging

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, f1_score, balanced_accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold 

from lazypredict.Supervised import LazyRegressor

import pandas
import mlflow
from mlflow import MlflowClient
import plotly.express as plt

data_path = "/home/kinged/Documents/shoes/datasets/shoes_main.csv"
data = pandas.read_csv(data_path, index_col=0)
data.sample(3)

,dos,prod_id,prod_name,sold_quantity,sold_price,total_sale,customer_id,store_loc,target_gender,customer_gender
5063,2019-06-30,P0016,Shoe Model 16,1,41.80,41.80,C7754,Chicago,Unisex,Female
8081,2022-09-24,P0002,Shoe Model 2,4,49.25,197.00,C9301,New York,Female,Unisex
3484,2017-10-20,P0017,Shoe Model 17,5,193.88,969.40,C1833,Chicago,Unisex,Unisex


lazypredict section


In [3]:
columns = ['prod_name', 'store_loc', 'target_gender', 'customer_gender']

def encoder(x: pandas.DataFrame| pandas.Series):
        return LabelEncoder().fit_transform(x)

for x in columns:
    data[x] = encoder(data[x])
    data[x] = data[x].astype('float64')

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9231 entries, 0 to 9230
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   dos              9231 non-null   object 
 1   prod_id          9231 non-null   object 
 2   prod_name        9231 non-null   int64  
 3   sold_quantity    9231 non-null   int64  
 4   sold_price       9231 non-null   float64
 5   total_sale       9231 non-null   float64
 6   customer_id      9231 non-null   object 
 7   store_loc        9231 non-null   int64  
 8   target_gender    9231 non-null   int64  
 9   customer_gender  9231 non-null   int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 793.3+ KB


In [7]:
prime_features_x = ['prod_name', 'store_loc', 'customer_gender', 'sold_quantity', 'sold_price']

x_prime = data[prime_features_x].values
y_prime = data['total_sale'].values

x, xtest, y, ytest = train_test_split(x_prime, y_prime, shuffle=True, test_size=0.3)
xtrain, xval, ytrain, yval = train_test_split(x, y, shuffle=True, test_size=0.1)

lazy = LazyRegressor(ignore_warnings=True,
                     random_state=45,
                     regressors='all', 
                     predictions=True)

models, predictions = lazy.fit(xtrain, xtest, ytrain, ytest)
models

100%|██████████| 42/42 [00:33<00:00,  1.27it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003723 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 296
[LightGBM] [Info] Number of data points in the train set: 5814, number of used features: 5
[LightGBM] [Info] Start training from score 575.337242


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
GaussianProcessRegressor,1.00,1.00,0.49,8.14
ExtraTreesRegressor,1.00,1.00,0.52,1.83
RandomForestRegressor,1.00,1.00,1.20,2.48
BaggingRegressor,1.00,1.00,1.65,0.28
DecisionTreeRegressor,1.00,1.00,2.05,0.07
ExtraTreeRegressor,1.00,1.00,2.68,0.06
XGBRegressor,1.00,1.00,4.05,0.26
LGBMRegressor,1.00,1.00,4.65,0.14
HistGradientBoostingRegressor,1.00,1.00,4.69,0.49


scikit-learn section

In [ ]:
bagging = BaggingRegressor()
rf = RandomForestRegressor()
dt = DecisionTreeRegressor()
extra_tree = ExtraTreeRegressor()

# train these models and tune their hyper parameters

feature extraction section

In [ ]:
models = [bagging, rf, dt, extra_tree]

def feature_extract(data, model):
    extractor = SelectFromModel(estimator=model.fit(x, y))    
    extractor.transform(data)
    columns = extractor.get_support(indices=True)
    main_feature_columns = data.feat

    # parquet_path = "/home/kinged/Documents/shoes/features/feature_repo/data/features_main.parquet"
    # pandas.DataFrame().to_parquet(parquet_path, "fastparquet")

    # feature_columns = data.columns.drop(labels=['prod_id', 'customer_id'])
    # features = data[feature_columns]
    
    # feast_path = "/home/kinged/Documents/shoes/features/feature_repo/data/features.parquet"
    # features.to_parquet(feast_path)

if __name__ == "__main__":
    feature_extract()

mlflow section

In [ ]:
mlflow.set_experiment(experiment_id='',
                      experiment_name='')

